In [10]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask_ml.xgboost import XGBRegressor
from dask_ml.preprocessing import DummyEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pathlib

In [11]:
path = pathlib.Path("./Data/Filtered/")
filepaths = list(path.iterdir())

In [12]:
my_df = dd.read_csv(filepaths, dtype={'engine_displacement': 'object',
       'horsepower': 'object'})
my_df.head()

,Unnamed: 0,vin,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,exterior_color,...,price,salvage,seller_rating,sp_name,theft_title,transmission,transmission_display,wheel_system,wheel_system_display,year
0,0,1FM5K8HC4LGA13788,SUV / Crossover,Grandville,18.0,406,49418,V6,3000.0,Agate Black,...,59677.0,False,4.466667,Borgman Ford Mazda,False,A,Automatic,AWD,All-Wheel Drive,2020
1,1,1C4RJEBT9DC539441,SUV / Crossover,Raleigh,False,48,27616,False,5700.0,Blue,...,13500.0,False,3.555556,Raleigh Pre-Owned,False,A,Automatic,4X2,4X2,2013
2,2,JTMBK31V475028962,SUV / Crossover,Indianapolis,19.0,26,46222,V6,3500.0,White,...,7900.0,False,2.333333,California Auto Sales,False,A,5-Speed Automatic,4WD,Four-Wheel Drive,2007
3,3,4T1G11AKXLU335390,Sedan,Durham,28.0,235,27707,I4,2500.0,Super White,...,27324.0,False,4.312500,Mark Jacobson Toyota,False,A,8-Speed Automatic,FWD,Front-Wheel Drive,2020
4,4,JH4KB16565C007356,Sedan,Raleigh,16.0,36,27604,V6,3500.0,GOLD,...,5900.0,False,4.571429,Brother Auto Sales,False,A,Automatic,AWD,All-Wheel Drive,2005


In [13]:
new_df = my_df.drop(columns=['Unnamed: 0'])
new_df.head()

,vin,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,exterior_color,frame_damaged,...,price,salvage,seller_rating,sp_name,theft_title,transmission,transmission_display,wheel_system,wheel_system_display,year
0,1FM5K8HC4LGA13788,SUV / Crossover,Grandville,18.0,406,49418,V6,3000.0,Agate Black,False,...,59677.0,False,4.466667,Borgman Ford Mazda,False,A,Automatic,AWD,All-Wheel Drive,2020
1,1C4RJEBT9DC539441,SUV / Crossover,Raleigh,False,48,27616,False,5700.0,Blue,False,...,13500.0,False,3.555556,Raleigh Pre-Owned,False,A,Automatic,4X2,4X2,2013
2,JTMBK31V475028962,SUV / Crossover,Indianapolis,19.0,26,46222,V6,3500.0,White,False,...,7900.0,False,2.333333,California Auto Sales,False,A,5-Speed Automatic,4WD,Four-Wheel Drive,2007
3,4T1G11AKXLU335390,Sedan,Durham,28.0,235,27707,I4,2500.0,Super White,False,...,27324.0,False,4.312500,Mark Jacobson Toyota,False,A,8-Speed Automatic,FWD,Front-Wheel Drive,2020
4,JH4KB16565C007356,Sedan,Raleigh,16.0,36,27604,V6,3500.0,GOLD,False,...,5900.0,False,4.571429,Brother Auto Sales,False,A,Automatic,AWD,All-Wheel Drive,2005


In [52]:
test = new_df.head(50000, npartitions=10)
my_df = pd.DataFrame(test)

In [53]:
my_df.to_csv('./Data/TestData.csv')

In [54]:
my_df = pd.read_csv('./Data/TestData.csv')
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            50000 non-null  int64  
 1   vin                   50000 non-null  object 
 2   body_type             50000 non-null  object 
 3   city                  50000 non-null  object 
 4   city_fuel_economy     50000 non-null  object 
 5   daysonmarket          50000 non-null  int64  
 6   dealer_zip            50000 non-null  int64  
 7   engine_cylinders      50000 non-null  object 
 8   engine_displacement   50000 non-null  object 
 9   exterior_color        50000 non-null  object 
 10  frame_damaged         50000 non-null  bool   
 11  franchise_dealer      50000 non-null  bool   
 12  franchise_make        50000 non-null  object 
 13  fuel_tank_volume      50000 non-null  object 
 14  fuel_type             50000 non-null  object 
 15  has_accidents      

In [15]:
my_df.head()

,vin,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,exterior_color,frame_damaged,...,price,salvage,seller_rating,sp_name,theft_title,transmission,transmission_display,wheel_system,wheel_system_display,year
0,1FM5K8HC4LGA13788,SUV / Crossover,Grandville,18.0,406,49418,V6,3000.0,Agate Black,False,...,59677.0,False,4.466667,Borgman Ford Mazda,False,A,Automatic,AWD,All-Wheel Drive,2020
1,1C4RJEBT9DC539441,SUV / Crossover,Raleigh,False,48,27616,False,5700.0,Blue,False,...,13500.0,False,3.555556,Raleigh Pre-Owned,False,A,Automatic,4X2,4X2,2013
2,JTMBK31V475028962,SUV / Crossover,Indianapolis,19.0,26,46222,V6,3500.0,White,False,...,7900.0,False,2.333333,California Auto Sales,False,A,5-Speed Automatic,4WD,Four-Wheel Drive,2007
3,4T1G11AKXLU335390,Sedan,Durham,28.0,235,27707,I4,2500.0,Super White,False,...,27324.0,False,4.312500,Mark Jacobson Toyota,False,A,8-Speed Automatic,FWD,Front-Wheel Drive,2020
4,JH4KB16565C007356,Sedan,Raleigh,16.0,36,27604,V6,3500.0,GOLD,False,...,5900.0,False,4.571429,Brother Auto Sales,False,A,Automatic,AWD,All-Wheel Drive,2005


In [50]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [17]:
my_df.loc[my_df['city_fuel_economy'] == 'False', 'city_fuel_economy'] = np.NaN
my_df.loc[my_df['engine_cylinders'] == 'False', 'engine_cylinders'] = np.NaN
my_df.loc[my_df['engine_displacement'] == 'False', 'engine_displacement'] = np.NaN
my_df.loc[my_df['body_type'] == 'False', 'body_type'] = np.NaN
my_df.loc[my_df['mileage'] == 0.0, 'mileage'] = np.NaN
my_df.loc[my_df['owner_count'] == 0.0, 'owner_count'] = np.NaN

In [18]:
not_needed_cols = ['wheel_system_display', 'transmission_display', 'sp_name',
                   'listing_color', 'listed_date', 'interior_color', 'exterior_color', 'vin']
cols_that_need_fixing = ['maximum_seating', 'engine_displacement', 'city_fuel_economy', 'fuel_tank_volume',
                        'highway_fuel_economy', 'horsepower']

not_needed_cols.extend(cols_that_need_fixing)

updated_df = my_df.drop(not_needed_cols, axis=1)

In [20]:
y = my_df.price
X = updated_df.drop(['price'], axis=1)

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [22]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

print(object_cols)

['body_type', 'city', 'engine_cylinders', 'franchise_make', 'fuel_type', 'make_name', 'model_name', 'transmission', 'wheel_system']


In [51]:
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

my_cols = categorical_cols + numerical_cols

X_train_edit = X_train[my_cols].copy()
X_valid_edit = X_valid[my_cols].copy()

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

model = RandomForestRegressor(n_estimators=150, random_state=1)

# model = XGBRegressor()

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_valid)

Score: 50693192.95438831
Mean of all prices: 29095.9675
